In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data=pd.read_csv('../data/res_mapping.csv', nrows=5000).dropna()
data['res_dt']=pd.to_datetime(data['res_dt'], format="%d.%m.%y %H:%M")
data['dep_soll']=pd.to_datetime(data['dep_soll'], format="%d.%m.%y %H:%M")
data['diff']=abs((data['res_dt']-data['dep_soll']).dt.total_seconds())
data['dep_soll_date']=data['dep_soll'].astype(str).str[:10]

In [ ]:
def weekday_to_number(day):
    if day == 'Mo':
        return 0
    elif day == 'Di':
        return 1
    elif day == 'Mi':
        return 2
    elif day == 'Do':
        return 3
    elif day == 'Fr':
        return 4
    elif day == 'Sa':
        return 5
    elif day == 'So':
        return 6

In [ ]:
train_mappings = pd.read_csv('../data/train_mapping.csv').dropna()

In [ ]:
for index, row in train_mappings.iterrows():
    vp = row['VP-Tagesleistung (Spaltte R)']
    
    if (len(vp) > 5):
        # Only support one or two day vp
        vp = 'Mo-So'
    days = vp.split('-')
    
    if len(days) == 1:
        train_mappings.at[index, 'VP-Tagesleistung (Spaltte R)'] = [weekday_to_number(days[0])]
    else:
        train_mappings.at[index, 'VP-Tagesleistung (Spaltte R)'] = list(range(weekday_to_number(days[0]), weekday_to_number(days[1])))
train_mappings

In [ ]:
block_mappings = pd.read_csv('../data/block_mapping.csv').dropna()

In [ ]:
line_stops=pd.read_csv('../data/line_stops.csv')
line_stops = line_stops[np.logical_not(np.logical_and(line_stops['Arrival time'].isnull(), line_stops['Departure time'].isnull()))]
line_stops['Stop time'] = line_stops['Arrival time'].fillna(line_stops['Departure time'])
line_stops['Stop time'] = pd.to_datetime(line_stops['Stop time'], format="%d.%m.%y %H:%M")
line_stops.drop(['Arrival time'], axis=1, inplace=True)
line_stops.drop(['Departure time'], axis=1, inplace=True)
line_stops

In [ ]:
opuic_mapping = pd.read_csv('../data/opuic_mapping.csv')
opuic_mapping

In [ ]:
data['stops'] = None
data.dtypes

In [ ]:
for index, row in data.iterrows():
    from_station = opuic_mapping[opuic_mapping['Station abbreviation'] == row['bp_from']].iloc[0, 0] #OPUIC
    to_station = opuic_mapping[opuic_mapping['Station abbreviation'] == row['bp_to']].iloc[0, 0]#OPUIC
    # Find all stops and sort by stop time
    stops = line_stops[line_stops['Linie'] == row['train_nr']].sort_values(by='Stop time')
    
    # Locate the indecies
    from_el = stops[stops['OPUIC'] == from_station]
    to_el = stops[stops['OPUIC'] == to_station]
    
    if len(from_el) < 1 or len(to_el) < 1:
        # Force stop this row, stop missing
        continue
    
    from_el = from_el['Stop time'].iloc[0]
    to_el = to_el['Stop time'].iloc[0]
    
    stops = stops.loc[(stops['Stop time'] >= from_el) & (stops['Stop time'] <= to_el)]['OPUIC'].tolist()
    
    if len(stops) < 1:
        print('SOMETHING WENT REALLY WRONG')
        
    data.at[index, 'stops'] = frozenset([stop for stop in stops])
    
    weekday = row['dep_soll'].weekday()
    block_nr = train_mappings.loc[
        np.logical_and(
            train_mappings['Zug'] == row['train_nr'], 
            weekday in train_mappings['VP-Tagesleistung (Spaltte R)'])].iloc[0, 3]
    
    found_block = block_mappings.loc[block_mappings['Block Bezeichnung in Jahresformation Fpl-2022'] == block_nr]
    
    if len(found_block) > 0:
        data.at[index, 'max_res_count'] = found_block.iloc[0, 1] 
    else:
        data.at[index, 'max_res_count'] = 0

In [ ]:
data

In [ ]:
for index, row in data.iterrows():
    reserves = data.loc[np.logical_and(
        data['train_nr'] == row['train_nr'],
        len(row['stops'].intersection(data['stops'])) > 0)]
    print(reserves)
    #data.at[index, 'res_count'] = len(data.loc[
    #    np.logical_and(data['train_nr'] == row['train_nr'], 
    #                   data['dep_soll_date'] == row['dep_soll_date'])])

In [ ]:
data

In [ ]:
data[data['res_count'] > data['max_res_count']]

In [ ]:
counts = data['max_res_count'] - data['res_count']
len(counts.where(counts <= 0).dropna())

In [ ]:
data.to_csv('../data/output.csv', columns = ['stops', 'diff', 'bp_to_token'], index=False)